In [1]:
import pandas as pd

rides = pd.read_parquet('../data/transformed/validated_rides_2022_01.parquet')

rides.head(20)

,pickup_datetime,pickup_location_id
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68
5,2022-01-01 00:40:15,138
6,2022-01-01 00:20:50,233
7,2022-01-01 00:13:04,238
8,2022-01-01 00:30:02,166
9,2022-01-01 00:48:52,236


In [2]:
rides['pickup_hour'] = rides['pickup_datetime'].dt.floor('H')
rides

,pickup_datetime,pickup_location_id,pickup_hour
0,2022-01-01 00:35:40,142,2022-01-01 00:00:00
1,2022-01-01 00:33:43,236,2022-01-01 00:00:00
2,2022-01-01 00:53:21,166,2022-01-01 00:00:00
3,2022-01-01 00:25:21,114,2022-01-01 00:00:00
4,2022-01-01 00:36:48,68,2022-01-01 00:00:00
...,...,...,...
2463926,2022-01-31 23:36:53,90,2022-01-31 23:00:00
2463927,2022-01-31 23:44:22,107,2022-01-31 23:00:00
2463928,2022-01-31 23:39:00,113,2022-01-31 23:00:00
2463929,2022-01-31 23:36:42,148,2022-01-31 23:00:00


In [3]:
agg_rides = rides.groupby(['pickup_hour', 'pickup_location_id']).size().reset_index()
agg_rides.rename(columns={0: 'rides'}, inplace=True)

agg_rides

,pickup_hour,pickup_location_id,rides
0,2022-01-01 00:00:00,4,11
1,2022-01-01 00:00:00,7,6
2,2022-01-01 00:00:00,10,1
3,2022-01-01 00:00:00,12,2
4,2022-01-01 00:00:00,13,12
...,...,...,...
66863,2022-01-31 23:00:00,261,4
66864,2022-01-31 23:00:00,262,8
66865,2022-01-31 23:00:00,263,26
66866,2022-01-31 23:00:00,264,24


In [25]:
from tqdm import tqdm

def add_missing_slots(agg_rides: pd.DataFrame) -> pd.DataFrame:
    
    location_ids = agg_rides['pickup_location_id'].unique()
    full_range = pd.date_range(
        agg_rides['pickup_hour'].min(), agg_rides['pickup_hour'].max(), freq='H')
    output = pd.DataFrame()
    for location_id in tqdm(location_ids):

        # keep only rides for this 'location_id'
        agg_rides_i = agg_rides.loc[agg_rides.pickup_location_id == location_id, ['pickup_hour', 'rides']]
            
        # quick way to add missing dates with 0 in a Series
        # taken from https://stackoverflow.com/a/19324591
        agg_rides_i.set_index('pickup_hour', inplace=True)
        agg_rides_i.index = pd.DatetimeIndex(agg_rides_i.index)
        agg_rides_i = agg_rides_i.reindex(full_range, fill_value=0)
        
        # add back `location_id` columns
        agg_rides_i['pickup_location_id'] = location_id

        output = pd.concat([output, agg_rides_i])
    
    # move the purchase_day from the index to a dataframe column
    output = output.reset_index().rename(columns={'index': 'pickup_hour'})
    
    return output

In [23]:
# agg_rides_indexed = agg_rides.set_index(['pickup_hour', 'pickup_location_id'])
# unique_location_ids = rides['pickup_location_id'].unique()

# min_time = agg_rides['pickup_hour'].min()
# max_time = agg_rides['pickup_hour'].max()
# select_times = pd.date_range(min_time, max_time, freq='H')

# agg_multi_index = pd.MultiIndex.from_product([select_times, unique_location_ids], names=['pickup_hour', 'pickup_location_id'])

# agg_final = agg_rides_indexed.reindex(agg_multi_index, fill_value=0).reset_index()
# agg_final

,pickup_hour,pickup_location_id,rides
0,2022-01-01 00:00:00,142,144
1,2022-01-01 00:00:00,236,104
2,2022-01-01 00:00:00,166,11
3,2022-01-01 00:00:00,114,106
4,2022-01-01 00:00:00,68,159
...,...,...,...
191203,2022-01-31 23:00:00,105,0
191204,2022-01-31 23:00:00,109,0
191205,2022-01-31 23:00:00,176,0
191206,2022-01-31 23:00:00,115,0


In [24]:
# agg_final[agg_final.rides == 0]

,pickup_hour,pickup_location_id,rides
88,2022-01-01 00:00:00,82,0
94,2022-01-01 00:00:00,223,0
95,2022-01-01 00:00:00,202,0
96,2022-01-01 00:00:00,228,0
97,2022-01-01 00:00:00,179,0
...,...,...,...
191203,2022-01-31 23:00:00,105,0
191204,2022-01-31 23:00:00,109,0
191205,2022-01-31 23:00:00,176,0
191206,2022-01-31 23:00:00,115,0


In [26]:
agg_rides_all_slots = add_missing_slots(agg_rides)

100%|██████████| 257/257 [00:00<00:00, 368.62it/s]


In [27]:
agg_rides_all_slots

,pickup_hour,rides,pickup_location_id
0,2022-01-01 00:00:00,11,4
1,2022-01-01 01:00:00,15,4
2,2022-01-01 02:00:00,26,4
3,2022-01-01 03:00:00,8,4
4,2022-01-01 04:00:00,9,4
...,...,...,...
191203,2022-01-31 19:00:00,0,176
191204,2022-01-31 20:00:00,0,176
191205,2022-01-31 21:00:00,0,176
191206,2022-01-31 22:00:00,0,176


In [28]:
agg_rides_all_slots[agg_rides_all_slots.rides == 0]

,pickup_hour,rides,pickup_location_id
8,2022-01-01 08:00:00,0,4
23,2022-01-01 23:00:00,0,4
25,2022-01-02 01:00:00,0,4
27,2022-01-02 03:00:00,0,4
28,2022-01-02 04:00:00,0,4
...,...,...,...
191203,2022-01-31 19:00:00,0,176
191204,2022-01-31 20:00:00,0,176
191205,2022-01-31 21:00:00,0,176
191206,2022-01-31 22:00:00,0,176


In [29]:
from typing import Optional, List
import plotly.express as px

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

def plot_rides(
    rides: pd.DataFrame,
    locations: Optional[List[int]] = None
    ):
    """
    Plot time-series data
    """
    rides_to_plot = rides[rides.pickup_location_id.isin(locations)] if locations else rides

    fig = px.line(
        rides_to_plot,
        x="pickup_hour",
        y="rides",
        color='pickup_location_id',
        template='none',
    )

    fig.show()

In [37]:
plot_rides(agg_rides_all_slots, locations=[43, 140])

In [ ]:
agg_rides_all_slots.to_parquet('../data/transformed/ts_data_2022_01.parquet')